In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
data = Path('/data/Datastore/Beijing') / 'pm2_5_df.h5'
data.exists()

True

In [4]:
df = pd.read_hdf(data, key='df')

In [5]:
df[0:20]

,No,datetime,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1.0,2013-03-01 00:00:00,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,1.0,2013-03-01 00:00:00,3.0,6.0,13.0,7.0,300.0,85.0,-2.3,1020.8,-19.7,0.0,E,0.5,Changping
2,1.0,2013-03-01 00:00:00,4.0,4.0,3.0,NaN,200.0,82.0,-2.3,1020.8,-19.7,0.0,E,0.5,Dingling
3,1.0,2013-03-01 00:00:00,9.0,9.0,3.0,17.0,300.0,89.0,-0.5,1024.5,-21.4,0.0,NNW,5.7,Dongsi
4,1.0,2013-03-01 00:00:00,4.0,4.0,14.0,20.0,300.0,69.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Guanyuan
5,1.0,2013-03-01 00:00:00,6.0,18.0,5.0,NaN,800.0,88.0,0.1,1021.1,-18.6,0.0,NW,4.4,Gucheng
6,1.0,2013-03-01 00:00:00,7.0,7.0,3.0,2.0,100.0,91.0,-2.3,1020.3,-20.7,0.0,WNW,3.1,Huairou
7,1.0,2013-03-01 00:00:00,5.0,14.0,4.0,12.0,200.0,85.0,-0.5,1024.5,-21.4,0.0,NNW,5.7,Nongzhanguan
8,1.0,2013-03-01 00:00:00,3.0,6.0,3.0,8.0,300.0,44.0,-0.9,1025.8,-20.5,0.0,NW,9.3,Shunyi
9,1.0,2013-03-01 00:00:00,6.0,6.0,4.0,8.0,300.0,81.0,-0.5,1024.5,-21.4,0.0,NNW,5.7,Tiantan


In [13]:
import numpy as np
item = df.iloc[2]['NO2']

In [17]:
np.isnan(item)

True

In [5]:
mask = (df['datetime'] >= datetime(year=2014, month=5, day=1)) & (df['datetime'] <= datetime(year=2015, month=4, day=30))

In [6]:
brits_df = df[mask]

In [7]:
training_masks = [
            (df['datetime'] >= datetime(year=2014, month=5, day=1)) & (
                    df['datetime'] < datetime(year=2014, month=6, day=1)),
            (df['datetime'] >= datetime(year=2014, month=7, day=1)) & (
                    df['datetime'] < datetime(year=2014, month=9, day=1)),
            (df['datetime'] >= datetime(year=2014, month=10, day=1)) & (
                    df['datetime'] < datetime(year=2014, month=12, day=1)),
            (df['datetime'] >= datetime(year=2015, month=1, day=1)) & (
                    df['datetime'] < datetime(year=2015, month=3, day=1)),
            (df['datetime'] >= datetime(year=2015, month=4, day=1)) & (
                    df['datetime'] <= datetime(year=2015, month=4, day=30))
        ]
test_masks = [
            (df['datetime'] >= datetime(year=2014, month=6, day=1)) & (
                    df['datetime'] < datetime(year=2014, month=7, day=1)),
            (df['datetime'] >= datetime(year=2014, month=9, day=1)) & (
                    df['datetime'] < datetime(year=2014, month=10, day=1)),
            (df['datetime'] >= datetime(year=2014, month=12, day=1)) & (
                    df['datetime'] < datetime(year=2015, month=1, day=1)),
            (df['datetime'] >= datetime(year=2015, month=3, day=1)) & (
                    df['datetime'] < datetime(year=2015, month=4, day=1))
        ]

In [8]:
training_df = []
for mask in training_masks:
    training_df.append(df[mask])
training_df = pd.concat(training_df)
training_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69708 entries, 122688 to 227531
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   No        69708 non-null  float64       
 1   datetime  69708 non-null  datetime64[ns]
 2   PM2.5     68313 non-null  float64       
 3   PM10      68727 non-null  float64       
 4   SO2       67967 non-null  float64       
 5   NO2       67386 non-null  float64       
 6   CO        68292 non-null  float64       
 7   O3        68418 non-null  float64       
 8   TEMP      69541 non-null  float64       
 9   PRES      69540 non-null  float64       
 10  DEWP      69539 non-null  float64       
 11  RAIN      69539 non-null  float64       
 12  wd        69708 non-null  object        
 13  WSPM      69539 non-null  float64       
 14  station   69708 non-null  object        
dtypes: datetime64[ns](1), float64(12), object(2)
memory usage: 8.5+ MB


In [9]:
training_df.isna().sum()

No             0
datetime       0
PM2.5       1395
PM10         981
SO2         1741
NO2         2322
CO          1416
O3          1290
TEMP         167
PRES         168
DEWP         169
RAIN         169
wd             0
WSPM         169
station        0
dtype: int64

In [10]:
test_df = []
for mask in test_masks:
    test_df.append(df[mask])
test_df = pd.concat(test_df)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35136 entries, 131616 to 219167
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   No        35136 non-null  float64       
 1   datetime  35136 non-null  datetime64[ns]
 2   PM2.5     33930 non-null  float64       
 3   PM10      34106 non-null  float64       
 4   SO2       33800 non-null  float64       
 5   NO2       33649 non-null  float64       
 6   CO        33880 non-null  float64       
 7   O3        33643 non-null  float64       
 8   TEMP      35132 non-null  float64       
 9   PRES      35132 non-null  float64       
 10  DEWP      35132 non-null  float64       
 11  RAIN      35130 non-null  float64       
 12  wd        35136 non-null  object        
 13  WSPM      35133 non-null  float64       
 14  station   35136 non-null  object        
dtypes: datetime64[ns](1), float64(12), object(2)
memory usage: 4.3+ MB


In [11]:
test_df.isna().sum()

No             0
datetime       0
PM2.5       1206
PM10        1030
SO2         1336
NO2         1487
CO          1256
O3          1493
TEMP           4
PRES           4
DEWP           4
RAIN           6
wd             0
WSPM           3
station        0
dtype: int64

In [12]:
from torch_geometric.data import Data

In [13]:
example_train = df[training_masks[0]]

In [14]:
example_train.iloc[0]

No                      10225.0
datetime    2014-05-01 00:00:00
PM2.5                     101.0
PM10                      160.0
SO2                        15.0
NO2                        78.0
CO                        600.0
O3                         85.0
TEMP                       17.7
PRES                     1002.3
DEWP                       13.2
RAIN                        0.0
wd                           SW
WSPM                        1.5
station            Aotizhongxin
Name: 122688, dtype: object

In [15]:
batch_size = 5

In [16]:
example_train.iloc[0:5]

,No,datetime,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
122688,10225.0,2014-05-01,101.0,160.0,15.0,78.0,600.0,85.0,17.7,1002.3,13.2,0.0,SW,1.5,Aotizhongxin
122689,10225.0,2014-05-01,90.0,136.0,12.0,44.0,900.0,120.0,18.0,998.9,9.9,0.0,N,0.2,Changping
122690,10225.0,2014-05-01,85.0,86.0,8.0,16.0,700.0,135.0,18.0,998.9,9.9,0.0,N,0.2,Dingling
122691,10225.0,2014-05-01,90.0,181.0,20.0,41.0,900.0,96.0,20.5,1003.9,11.9,0.0,WSW,1.9,Dongsi
122692,10225.0,2014-05-01,103.0,164.0,15.0,46.0,700.0,123.0,17.7,1002.3,13.2,0.0,SW,1.5,Guanyuan


In [17]:
x = []  # [num_nodes, num_node_features]
edge_index = []  # Graph connectivity in COO format with shape [2, num_edges] and type torch.long
y = []  # Target to train against


In [76]:
import pydantic
from datetime import datetime
import torch
from torch import FloatTensor, LongTensor, BoolTensor
from pydantic import BaseModel
from typing import Optional

class TargetNode(BaseModel):
    features: FloatTensor
    type_index: LongTensor
    time: FloatTensor
    spatial_index: LongTensor

    class Config:
        arbitrary_types_allowed = True

class ContinuousTimeGraphSample(BaseModel):
    node_features: FloatTensor
    edge_index: Optional[LongTensor]
    attention_mask: Optional[BoolTensor]
    time: FloatTensor
    target: TargetNode
    type_index: LongTensor
    spatial_index: LongTensor
    category_index: Optional[LongTensor]

    class Config:
        arbitrary_types_allowed = True


In [19]:
pd.unique(df['wd'])

array(['NNW', 'E', 'NW', 'WNW', 'N', 'ENE', 'NNE', 'W', 'NE', 'SSW',
       'ESE', 'SE', 'S', 'SSE', 'SW', 'WSW', 'None'], dtype=object)

In [20]:
pd.unique(df['station'])

array(['Aotizhongxin', 'Changping', 'Dingling', 'Dongsi', 'Guanyuan',
       'Gucheng', 'Huairou', 'Nongzhanguan', 'Shunyi', 'Tiantan',
       'Wanliu', 'Wanshouxigong'], dtype=object)

In [67]:
import random
target_index = random.randint(12*11*2+1, 12*11*3-1)
print(target_index)
set = target_index//11
print(set)
sample = example_train.iloc[set]
print(sample[features[target_index - set*11]])
sample

338
30
11.7


No                      10227.0
datetime    2014-05-01 02:00:00
PM2.5                     102.0
PM10                      128.0
SO2                         NaN
NO2                        51.0
CO                          NaN
O3                         55.0
TEMP                       16.1
PRES                      999.4
DEWP                       11.7
RAIN                        0.0
wd                          NNW
WSPM                        1.0
station                 Huairou
Name: 122718, dtype: object

In [77]:

features = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
time = 'datetime'
category = 'wd'
spatial = 'station'
unique_wd = ['NNW', 'E', 'NW', 'WNW', 'N', 'ENE', 'NNE', 'W', 'NE', 'SSW', 'ESE', 'SE', 'S', 'SSE', 'SW', 'WSW', 'None']
unique_stations = ['Aotizhongxin', 'Changping', 'Dingling', 'Dongsi', 'Guanyuan', 'Gucheng', 'Huairou', 'Nongzhanguan', 'Shunyi', 'Tiantan', 'Wanliu', 'Wanshouxigong']
node_features = []
time_index = []
type_index = []
spatial_index = []
category_index = []
time_start = None
target = {}

for i in range(12*3):
    sample = example_train.iloc[i]
    if time_start is None:
        time_start = example_train.iloc[12*3-1][time]
    tau = (time_start - sample[time]).seconds
    for j, key in enumerate(features):
        if i*11 + j != target_index:
            type_index.append(j)
            node_features.append(sample[key])
            spatial_index.append(unique_stations.index(sample[spatial]))
            category_index.append(unique_wd.index(sample[category]))
            time_index.append(tau)
        else:
            target['features'] = torch.tensor([sample[key]], dtype=torch.float)
            target["type_index"] = torch.tensor([j], dtype=torch.long)
            target["spatial_index"] = torch.tensor([unique_stations.index(sample[spatial])], dtype=torch.long)
            target["category_index"] = torch.tensor([unique_wd.index(sample[category])], dtype=torch.long)
            target["time"] = torch.tensor([tau], dtype=torch.float)
print(target)

{'features': tensor([11.7000]), 'type_index': tensor([8]), 'spatial_index': tensor([6]), 'category_index': tensor([0]), 'time': tensor([0.])}


In [78]:
graph = {
    "node_features": torch.tensor(node_features, dtype=torch.float),
    'time': torch.tensor(time_index, dtype=torch.float),
    "type_index": torch.tensor(type_index, dtype=torch.long),
    "spatial_index": torch.tensor(spatial_index, dtype=torch.long),
    "category_index": torch.tensor(category_index, dtype=torch.long),
    "target": TargetNode(**target)
}
graph['attention_mask'] = graph['time'].unsqueeze(-1).T < graph['time'].unsqueeze(-1)


In [80]:
torch.save(ContinuousTimeGraphSample(**graph), "../../AGG/test_data/sample.pt")

In [64]:
now = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
print(len(now))

132


In [51]:
time = torch.tensor(time_index)

In [57]:
time2 = time.unsqueeze(-1).T >= time.unsqueeze(-1)
time3 = time.unsqueeze(-1).T < time.unsqueeze(-1)

In [58]:
time3

tensor(True)

In [53]:
time2.float()

tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [60]:
data = [0, 0, 10, 10, 20, 20]
data.reverse()
time_test = torch.tensor(data)

In [61]:
data

[20, 20, 10, 10, 0, 0]

In [63]:
(time_test.unsqueeze(-1).T < time_test.unsqueeze(-1))

tensor([[False, False,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False]])

SyntaxError: EOL while scanning string literal (<ipython-input-38-3f8cd4c64e06>, line 1)